In [3]:
import yaml
import requests
from bs4 import BeautifulSoup
from pymed import PubMed
import urllib.request
import requests
import gdown
import os
import pandas as pd
from PIL import Image


### Publications

In [58]:
url = 'https://weissmanlab.ucsf.edu/publications/publications.html'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

old_website_pubs = [(a.get('href'), a.text) for pub_div in soup.find_all(id='publication') for a in pub_div.find_all('a')]

In [2]:
pubmed = PubMed(tool="MyTool", email="lenail@mit.edu")
results = list(pubmed.query("Jonathan Weissman", max_results=500))
len(results)

225

In [5]:
pubmed_pubs = []

for result in results:
    try:
        pubmed_pubs.append(
            dict(title=result.title,
                authors=[str(author['firstname'])+' '+str(author['lastname']) for author in result.authors],
                publication_date=result.publication_date.strftime("%d/%m/%y"),
                publication_year=result.publication_date.strftime("%Y"),
                pubmed_id=result.pubmed_id.split('\n')[0],
                abstract=result.abstract,
                doi=result.doi.split('\n')[0],
                journal=result.journal))

    except:
        # print(result.toDict())

        print(result.title)
        print(result.authors)
        print(result.publication_date)
        print(result.pubmed_id)
        print(result.abstract)
        print(result.doi)
        print(result.journal)

In [10]:
replacements = {
    'bioRxiv : the preprint server for biology': 'biorxiv',
    'Methods in molecular biology (Clifton, N.J.)': 'Methods in molecular biology',
    'Current biology : CB': 'Current biology',
    'Science (New York, N.Y.)': 'Science',
    'Proceedings of the National Academy of Sciences of the United States of America': 'PNAS',
    'Molecular & cellular proteomics : MCP': 'Molecular & cellular proteomics'
}

In [17]:
for p in pubmed_pubs:
    if p['journal'] in replacements:
        p['journal'] = replacements[p['journal']]

In [19]:
open('_data/publications.yml', 'w').write(yaml.dump(pubmed_pubs, default_flow_style=False, sort_keys=False))

349773

### Previous Site

In [3]:
people_url = 'https://weissmanlab.ucsf.edu/people/people.html'
alum_url = 'https://weissmanlab.ucsf.edu/people/alumni.html'
people_page = requests.get(people_url)
alum_page = requests.get(alum_url)

people_soup = BeautifulSoup(people_page.text, 'html.parser')
alum_soup = BeautifulSoup(alum_page.text, 'html.parser')

def get_text(elem, id):
    elem = elem.find(id=id)
    if elem: return elem.get_text(separator='|')
    else: return ''

people = [dict(name=get_text(person, 'peo_name').replace('|', ' '),
           pos=get_text(person, 'peo_pos'),
           email=get_text(person, 'peo_email'),
           img=person.find('img').get('src')) for person in people_soup.find_all(id='peo_grid')]
alums = [dict(name=get_text(person, 'peo_name').replace('|', ' '),
          pos=get_text(person, 'alum_pos'),
          now=get_text(person, 'alum_current_2line'),
          img=(person.find('img').get('src') if person.find('img') else None)) for person in alum_soup.find_all(id='peo_grid')][1:]

for p in people: p['first_name'] = p['name'].split(' ')[0]
for p in people: p['last_name'] = p['name'].split(' ')[-1]
for p in alums: p['first_name'] = p['name'].split(' ')[0]
for p in alums: p['last_name'] = p['name'].split(' ')[-1]

for p in people: p['alum'] = 'false'
for p in alums: p['alum'] = 'true'

In [4]:
for person in people:
    urllib.request.urlretrieve('https://weissmanlab.ucsf.edu/people/'+person['img'], 'assets/img/people/'+person['img'])

In [7]:
for person in alums:
    urllib.request.urlretrieve('https://weissmanlab.ucsf.edu/people/'+person['img'], 'assets/img/people/'+person['img'].split('/')[-1])

    person['img'] = person['img'].split('/')[-1]

In [8]:
people = people + alums
for p in people: p['id'] = p['first_name']+p['last_name']

In [9]:
def rename_img(p):
    if p['img'] == 'weibezahn.jpg' or p['img'] == '':
        p['img'] = ''
    else:
        change_from = f'assets/img/people/{p["img"]}'
        change_to = f'assets/img/people/{p["id"]+"."+p["img"].split(".")[-1]}'
        if os.path.exists(change_from): os.rename(change_from, change_to)
        p["img"] = change_to

for p in people: rename_img(p)

In [13]:
pd.DataFrame(people).to_csv('old_site_people.csv', index=False)

### Survey

In [15]:
people = pd.read_csv('old_site_people.csv')

In [29]:
survey_responses = pd.read_csv('https://docs.google.com/spreadsheets/d/1uxl0DQpqHvNdgI92UVx1Vyt4TQhkgt74cPp1dEn3Gjw'+'/export?format=csv')

In [30]:
survey_responses = survey_responses.rename(columns={'Name':'name', 'Role':'pos', 'Email':'email', 'Portrait / head shot':'img'})[['name', 'pos', 'email', 'img']]

In [32]:
survey_responses['first_name'] = survey_responses['name'].str.split(' ').str[0]
survey_responses['last_name'] = survey_responses['name'].str.split(' ').str[-1]
survey_responses['id'] = survey_responses['first_name']+survey_responses['last_name']
survey_responses['alum'] = False

In [46]:
for id, url in survey_responses.set_index('id')['img'].items():
    print(id, url)
    if type(url) == str:
        change_to = 'assets/img/people/'+id+'.jpg'
        gdown.download("https://drive.google.com/uc?id={}".format(url.split('id=')[-1]), change_to, quiet=False)

RichardShe https://drive.google.com/open?id=1pAqQi52Pr1B0xIEtMr3XaAas1hVMW08w


Downloading...
From: https://drive.google.com/uc?id=1pAqQi52Pr1B0xIEtMr3XaAas1hVMW08w
To: /Users/alex/Documents/weissmanlab/assets/img/people/RichardShe.jpg
100%|██████████| 588k/588k [00:00<00:00, 15.7MB/s]


LakshmiMiller-Vedam https://drive.google.com/open?id=1pAkanvnqJmd1D9i7Onl_6_lWLmg5nugY


Downloading...
From: https://drive.google.com/uc?id=1pAkanvnqJmd1D9i7Onl_6_lWLmg5nugY
To: /Users/alex/Documents/weissmanlab/assets/img/people/LakshmiMiller-Vedam.jpg
100%|██████████| 95.3k/95.3k [00:00<00:00, 3.84MB/s]


LukeKoblan https://drive.google.com/open?id=1IsF8g-d_w7A4XTUmI_2t21q2jGv0EKTf


...

YuanchengLu https://drive.google.com/open?id=1uH4d9gunYYbGys_pbbOdjgG-ZFjv8Zyx


Downloading...
From: https://drive.google.com/uc?id=1uH4d9gunYYbGys_pbbOdjgG-ZFjv8Zyx
To: /Users/alex/Documents/weissmanlab/assets/img/people/YuanchengLu.jpg
100%|██████████| 266k/266k [00:00<00:00, 6.95MB/s]


KatieYost https://drive.google.com/open?id=15BWQLjHXdd5f-6h_klvKl87e1t4Kx0gU


Downloading...
From: https://drive.google.com/uc?id=15BWQLjHXdd5f-6h_klvKl87e1t4Kx0gU
To: /Users/alex/Documents/weissmanlab/assets/img/people/KatieYost.jpg
100%|██████████| 742k/742k [00:00<00:00, 10.5MB/s]

JeffHussmann nan


In [57]:
set(people.id) & set(survey_responses.id)

{'JeffHussmann',
 'JosephReplogle',
 'KaterinaPopova',
 'LakshmiMiller-Vedam',
 'MattJones',
 'ReubenSaunders',
 'RichardShe',
 'XiaojieQiu'}

In [58]:
people = pd.concat((survey_responses, people)).drop_duplicates(subset=['id'], keep='first')

In [61]:
people = people.drop('img', axis=1)

In [63]:
people.to_csv('old_site_and_survey_people.csv', index=False)

In [64]:
pd.DataFrame(people).to_clipboard()

### Trim down lab photos to max size

In [2]:
import os
for img_path in os.listdir('assets/img/people/'):
    try:
        image = Image.open('assets/img/people/'+img_path)
        image.thumbnail((400, 400))
        image.save('assets/img/people/'+img_path)
    except:
        print(img_path)

.DS_Store


### Update People from spreadsheet

In [1]:
# id must be both the publication first/last, and the name of the image. Name is just the display name.

In [19]:
url = 'https://docs.google.com/spreadsheets/d/1-Eju9h1XovEBoBv0DGpxh92GYsZ8bzGkxRdRgUb7hvg'
people = pd.read_csv(url+'/export?format=csv')
# strip strings
df_obj = people.select_dtypes(['object'])
people[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [20]:
with open('_data/people.yml', 'r') as f:
    old_people_data = pd.json_normalize(yaml.safe_load(f))

#### New IDs:

In [22]:
people[ ~people.id.isin(old_people_data.id)]


,name,pos,email,id,alum,now
24,Joseph Min,Grad student,kmin@wi.mit.edu,JosephMin,False,NaN
25,Karina Smolyar,Grad student,ksmolyar@wi.mit.edu,KarinaSmolyar,False,NaN
32,Jeff Quinn,Post-doc,NaN,JeffreyQuinn,True,NaN


#### Updates to existing IDs:

In [33]:
people['update'] = 'current'
old_people_data['update'] = 'previous'

pd.concat([people[people.id.isin(old_people_data.id)],old_people_data[old_people_data.id.isin(people.id)]]).drop_duplicates(subset=['name'], keep=False).sort_values(by='id')


,name,pos,email,id,alum,now,update
10,Jeff Hussmann,Post-doc,jah@wi.mit.edu,JeffreyHussmann,False,NaN,current
10,Jeffrey Hussmann,Post-doc,jah@wi.mit.edu,JeffreyHussmann,False,NaN,previous
81,Matthew Larson,Post-doc,NaN,MatthewLarson,True,Scientist |Illumina Medical Laboratory,current
79,Matthew Larson,Post-doc,NaN,MatthewLarson,True,Scientist |Illumina Medical Laboratory,previous
6,Zeb Levine,Post-doc,levinez@wi.mit.edu,ZebulonLevine,False,NaN,current
6,Zebulon Levine,Post-doc,levinez@wi.mit.edu,ZebulonLevine,False,NaN,previous


#### Removal of IDs:

In [23]:
old_people_data[ ~old_people_data.id.isin(people.id)]

,name,pos,email,id,alum,now
30,Jeff Quinn,Post-doc,NaN,JeffQuinn,True,NaN
109,Kimberly Tipton,Grad student,NaN,KimberlyTipton,True,Scientist |CytomX Therapeutics


#### Write updated sheet to yaml

In [35]:
people = people.drop('update', axis=1)
people_records = [{k:v for k,v in m.items() if pd.notnull(v)} for m in people.to_dict(orient='records')]


In [41]:
class MyDumper(yaml.SafeDumper):
    # HACK: insert blank lines between top-level objects
    # inspired by https://stackoverflow.com/a/44284819/3786245
    def write_line_break(self, data=None):
        super().write_line_break(data)

        if len(self.indents) == 1:
            super().write_line_break()

In [ ]:
open('_data/people.yml', 'w').write(yaml.dump(people_records, Dumper=MyDumper, default_flow_style=False, sort_keys=False))
